# Multi-fidelity BO
Here we demonstrate how Multi-Fidelity Bayesian Optimization can be used to reduce
the computational cost of optimization by using lower fidelity surrogate models. The
goal is to learn functional dependance of the objective on input variables at low
fidelities (which are cheap to compute) and use that information to quickly find the
best objective value at higher fidelities (which are more expensive to compute). This
assumes that there is some learnable correlation between the objective values at
different fidelities.

Xopt implements the MOMF (https://botorch.org/tutorials/Multi_objective_multi_fidelity_BO)
algorithm which can be used to solve both single (this notebook) and multi-objective
(see multi-objective BO section) multi-fidelity problems. Under the hood this
algorithm attempts to solve a multi-objective optimization problem, where one
objective is the function objective and the other is a simple fidelity objective,
weighted by the ```cost_function``` of evaluating the objective at a given fidelity.

In [ ]:
from xopt.generators.bayesian import MultiFidelityGenerator
from xopt import Evaluator, Xopt
from xopt import VOCS
import os

import matplotlib.pyplot as plt
import numpy as np
import math

import pandas as pd


# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")

SMOKE_TEST = os.environ.get("SMOKE_TEST")
N_MC_SAMPLES = 1 if SMOKE_TEST else 128
N_RESTARTS = 1 if SMOKE_TEST else 20


def test_function(input_dict):
    x = input_dict["x"]
    s = input_dict["s"]
    return {"f": np.sin(x + (1.0 - s)) * np.exp((-s + 1) / 2)}


# define vocs


vocs = VOCS(
    variables={
        "x": [0, 2 * math.pi],
    },
    objectives={"f": "MINIMIZE"},
)

## plot the test function in input + fidelity space


In [ ]:
test_x = np.linspace(*vocs.bounds, 1000)
fidelities = [0.0, 0.5, 1.0]

fig, ax = plt.subplots()
for ele in fidelities:
    f = test_function({"x": test_x, "s": ele})["f"]
    ax.plot(test_x, f, label=f"s:{ele}")

ax.legend()

In [ ]:
# create xopt object
# get and modify default generator options
generator = MultiFidelityGenerator(vocs=vocs)

# specify a custom cost function based on the fidelity parameter
generator.cost_function = lambda s: s + 0.001

generator.numerical_optimizer.n_restarts = N_RESTARTS
generator.n_monte_carlo_samples = N_MC_SAMPLES

# pass options to the generator
evaluator = Evaluator(function=test_function)

X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator)
X

In [ ]:
# evaluate initial points at mixed fidelities to seed optimization
X.evaluate_data(
    pd.DataFrame({"x": [math.pi / 4, math.pi / 2.0, math.pi], "s": [0.0, 0.25, 0.0]})
)

In [ ]:
# get the total cost of previous observations based on the cost function
X.generator.calculate_total_cost()

In [ ]:
# run optimization until the cost budget is exhausted
# we subtract one unit to make sure we don't go over our eval budget
budget = 10
while X.generator.calculate_total_cost() < budget - 1:
    X.step()
    print(
        f"n_samples: {len(X.data)} "
        f"budget used: {X.generator.calculate_total_cost():.4} "
        f"hypervolume: {X.generator.calculate_hypervolume():.4}"
    )

In [ ]:
X.data

## Plot the model prediction and acquisition function inside the optimization space

In [ ]:
fig, ax = X.generator.visualize_model()

## Plot the Pareto front

In [ ]:
X.data.plot(x="f", y="s", style="o-")

In [ ]:
X.data

In [ ]:
# get optimal value at max fidelity, note that the actual maximum is 4.71
X.generator.get_optimum().to_dict()